<a href="https://colab.research.google.com/github/Elwing-Chou/tibame0410/blob/main/tibame_0428_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import urllib.request as req
import bs4 as bs
url = "https://24h.pchome.com.tw/search/?q=%E5%90%B9%E9%A2%A8%E6%A9%9F"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())
links = html.find_all("a", {"class":"c-prodInfoV2__link"})
for link in links:
    link_href = "https://24h.pchome.com.tw" + link["href"]
    print(link_href)

https://24h.pchome.com.tw/prod/DMAH47-A900H012R
https://24h.pchome.com.tw/prod/DDBH8B-A900GK0UE
https://24h.pchome.com.tw/prod/DDDU04-A900GKQ4T
https://24h.pchome.com.tw/prod/DMAH7M-A900HPUFB
https://24h.pchome.com.tw/prod/DDDU04-1900I79JH
https://24h.pchome.com.tw/prod/DMAHBY-A900I0K1F
https://24h.pchome.com.tw/prod/DMAHCW-A900IGQBP
https://24h.pchome.com.tw/prod/DMAH4C-A900I7YVV
https://24h.pchome.com.tw/prod/DMAH7C-A900HQBQH
https://24h.pchome.com.tw/prod/DMAH4C-A900I7YVH
https://24h.pchome.com.tw/prod/DMAH02-A900AN6LF
https://24h.pchome.com.tw/prod/DMAH02-A900ARLYE
https://24h.pchome.com.tw/prod/DMAH4C-A900HTGKT
https://24h.pchome.com.tw/prod/DMAH02-A900FUEMV
https://24h.pchome.com.tw/prod/DMAH7S-A90092JRB
https://24h.pchome.com.tw/prod/DMAH02-A900GN0FB
https://24h.pchome.com.tw/prod/DMAH7S-A900GDOA8
https://24h.pchome.com.tw/prod/DMAH4E-A9007XOV1
https://24h.pchome.com.tw/prod/DYAN6U-A900GGFEZ
https://24h.pchome.com.tw/prod/DMAH02-A900GAYQW
https://24h.pchome.com.tw/prod/DMAH02-A9

In [ ]:
# 名字
# 原價/目前售價
# 滿額贈是否有(Y/N)
# 進去以後
# 評價幾分/幾則
# 介紹
# 圖片的網址